# Decorators

In [1]:
from time import sleep, time

def f():
    sleep(.3)

def g():
    sleep(.5)

t = time()
f()
print('f took: ', time() - t) # f took: 0.3003859519958496

t = time()
g()
print('g took:', time() - t)  # g took: 0.5005719661712646

f took:  0.30155110359191895
g took: 0.5018911361694336


In [2]:
from time import sleep, time

def f():
    sleep(.3)

def g():
    sleep(.5)

def measure(func):
    t = time()
    func()
    print(func.__name__, 'took:', time() - t)

measure(f)  # f took: 0.30041074752807617
measure(g)  # g took: 0.5006198883056641

f took: 0.30518293380737305
g took: 0.5045621395111084


In [3]:
from time import sleep, time

def f(sleep_time=0.1): 
    sleep(sleep_time)

def measure(func, *args, **kwargs):
    t = time()
    func(*args, **kwargs)
    print(func.__name__, 'took:', time() - t)

measure(f, sleep_time=0.3) # f took: 0.3004162311553955
measure(f, 0.2) # f took: 0.20028162002563477

f took: 0.3005850315093994
f took: 0.20444798469543457


In [5]:
from time import sleep, time

def f(sleep_time=0.1):
    sleep(sleep_time)

def measure(func):
    def wrapper(*args, **kwargs):
        t = time()
        func(*args, **kwargs)
        print(func.__name__, 'took:', time() - t)
    return wrapper 

f = measure(f)  # decoration point

f(0.2)  # f took: 0.2002875804901123
f(sleep_time=0.3)  # f took: 0.3003721237182617
print(f.__name__)  # wrapper  <- ouch!

f took: 0.20204496383666992
f took: 0.3000967502593994
wrapper


In [6]:
from time import sleep, time

def measure(func):
    def wrapper(*args, **kwargs):
        t = time()
        func(*args, **kwargs)
        print(func.__name__, 'took:', time() - t)
    return wrapper 


@measure
def f(sleep_time=0.1):
    sleep(sleep_time)
    
f(0.2)  # f took: 0.2002875804901123
f(sleep_time=0.3)  # f took: 0.3003721237182617
print(f.__name__)  # wrapper  <- ouch!

f took: 0.20047688484191895
f took: 0.30260300636291504
wrapper


In [13]:
from time import sleep, time
from functools import wraps

def measure(func):
    #@wraps 데코레이터를 통해 데코레이터를 쓰지만 __name__, __doc__ 등은 f함수에 적용되도록 처리 
    @wraps(func)
    def wrapper(*args, **kwargs):
        t = time()
        func(*args, **kwargs)
        print(func.__name__, 'took:', time() - t)
    return wrapper

@measure
def f(sleep_time=0.1):
    """I'm a cat. I love to sleep! """
    sleep(sleep_time)

f(sleep_time=0.3)  # f took: 0.30039525032043457
print(f.__name__, ':', f.__doc__) # f : I'm a cat. I love to sleep!

f took: 0.30457496643066406
f : I'm a cat. I love to sleep! 


In [17]:
from time import sleep, time
from functools import wraps

def measure(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        t = time()
        result = func(*args, **kwargs) 
        print(func.__name__, 'took:', time() - t) 
        return result
    return wrapper

def max_result(func):
    @wraps(func)
    def wrapper(*args, **kwargs): 
        result = func(*args, **kwargs) 
        if result > 100:
            print('Result is too big ({0}). Max allowed is 100.'
                  .format(result))
        return result
    return wrapper

@measure
@max_result
def cube(n):
    return n ** 3
   
print(cube(2))
print("=================")
print(cube(5))

print("=================")
# @wraps만 써주면 여러번 decorator를 중첩으로 적용해도 __name__은 원함수에 잘 적용됨 
print(cube.__name__)

cube took: 3.0994415283203125e-06
8
Result is too big (125). Max allowed is 100.
cube took: 3.1948089599609375e-05
125
cube


# Decorator factory

In [28]:
# 원래 decorator는 함수를 인자로 받는데, 
# outer에는 넣고자 하는 변수를 인자로 하고, 그 안에 inner 함수를 정의하여 함수를 인자로 받도록 처리하면 
# 위 사례와 달리 인자를 받는 decorator 구현 가능 (dynamically하게 threshold 설정 가능)
# "closure" 라는 메커니즘을 사용한 것 

from functools import wraps
def max_result(threshold): 
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            if result > threshold:
                print(
                    'Result is too big ({0}). Max allowed is {1}.'
                    .format(result, threshold))
            return result
        return wrapper
    return decorator


@max_result(75) 
def cube(n):
    return n ** 3

@max_result(100) 
def square(n):
    return n ** 2

@max_result(1000) 
def multiply(a, b):
    return a * b

print(cube(5))
print(square(12))
print(multiply(1000, 20))

Result is too big (125). Max allowed is 75.
125
Result is too big (144). Max allowed is 100.
144
Result is too big (20000). Max allowed is 1000.
20000
